In [5]:
import cv2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame,"Student",(x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),2)

    cv2.imshow("Attendance Camera", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import sqlite3
from datetime import datetime
from tkinter import *

# ---------------- DATABASE SETUP ----------------
def init_db():
    conn = sqlite3.connect("attendance.db")
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS attendance (
        student_id INTEGER,
        name TEXT,
        date TEXT,
        time_in TEXT,
        time_out TEXT,
        reason TEXT
    )
    """)
    conn.commit()
    conn.close()

init_db()

STUDENT_ID = 101
STUDENT_NAME = "Student_1"

# ---------------- ENTRY FUNCTION ----------------
def mark_entry():
    conn = sqlite3.connect("attendance.db")
    cur = conn.cursor()

    date = datetime.now().strftime("%d-%m-%Y")
    time_in = datetime.now().strftime("%H:%M:%S")

    cur.execute("""
    INSERT INTO attendance VALUES (?,?,?,?,?,?)
    """, (STUDENT_ID, STUDENT_NAME, date, time_in, "-", "-"))

    conn.commit()
    conn.close()
    print("✅ ENTRY MARKED")

# ---------------- EXIT FUNCTION ----------------
def save_exit(reason):
    conn = sqlite3.connect("attendance.db")
    cur = conn.cursor()

    time_out = datetime.now().strftime("%H:%M:%S")

    cur.execute("""
    UPDATE attendance
    SET time_out=?, reason=?
    WHERE student_id=? AND time_out='-'
    """, (time_out, reason, STUDENT_ID))

    conn.commit()
    conn.close()
    print("🚪 EXIT MARKED WITH REASON")

def exit_popup():
    def submit():
        reason = entry.get()
        save_exit(reason)
        win.destroy()

    win = Tk()
    win.title("Exit Reason")

    Label(win, text="Enter Exit Reason").pack(pady=5)
    entry = Entry(win, width=30)
    entry.pack(pady=5)

    Button(win, text="Submit", command=submit).pack(pady=10)
    win.mainloop()

# ---------------- FACE DETECTION ----------------
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

print("🎥 Camera Started")
print("Press E → Entry")
print("Press X → Exit with Reason")
print("Press ESC → Quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, "Face Detected", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("Attendance System", frame)

    key = cv2.waitKey(1)

    if key == 27:  # ESC
        break
    elif key == ord('e') or key == ord('E'):
        mark_entry()
    elif key == ord('x') or key == ord('X'):
        exit_popup()

cap.release()
cv2.destroyAllWindows()


🎥 Camera Started
Press E → Entry
Press X → Exit with Reason
Press ESC → Quit


In [ ]:
if key == 27:  # ESC
    break
